In [ ]:
import os
from keras.models import Model
from keras.layers import Input, Dense, ReLU, Dropout, Softmax, Conv2D, MaxPool2D, Lambda, GaussianNoise
from keras.layers import Bidirectional, Flatten, CuDNNGRU
from keras.utils.vis_utils import plot_model

# Define ICAMC model (custom CNN architecture)
def ICAMC(weights=None,
          input_shape=[2,1024],
          classes=26,
          **kwargs):

    # Check if weights are valid
    if weights is not None and not (os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), '
                         'or the path to the weights file to be loaded.')

    dr = 0.45   # Dropout rate for regularization

    # Input layer
    input = Input(input_shape + [1], name='input')

    # First convolution layer + pooling
    x = Conv2D(32, (1, 8), activation="relu", name="conv1",
               padding='same', kernel_initializer='glorot_uniform')(input)
    x = MaxPool2D(pool_size=(2, 2))(x)

    # Second convolution layer
    x = Conv2D(32, (1, 4), activation="relu", name="conv2",
               padding='same', kernel_initializer='glorot_uniform')(x)

    # Third convolution layer
    x = Conv2D(64, (1, 8), activation="relu", name="conv3",
               padding='same', kernel_initializer='glorot_uniform')(x)
    x = Dropout(dr)(x)   # Dropout for regularization

    # Fourth convolution layer
    x = Conv2D(64, (1, 8), activation="relu", name="conv4",
               padding='same', kernel_initializer='glorot_uniform')(x)
    x = Dropout(dr)(x)   # Dropout again

    # Flatten convolution outputs for dense layers
    x = Flatten()(x)

    # Fully connected dense layer
    x = Dense(64, activation='relu', name='dense1')(x)
    x = Dropout(dr)(x)   # Dropout

    # Add Gaussian noise for robustness against overfitting
    x = GaussianNoise(1)(x)

    # Final classification layer (softmax for multiclass output)
    x = Dense(classes, activation='softmax', name='dense2')(x)

    # Build model
    model = Model(inputs=input, outputs=x)

    # Load pre-trained weights if provided
    if weights is not None:
        model.load_weights(weights)

    return model


# Run only when executed directly
import keras
if __name__ == '__main__':
    # Create the model
    model = ICAMC(None, input_shape=[2,1024], classes=26)

    # Compile with Adam optimizer & categorical crossentropy loss
    adam = keras.optimizers.Adam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999,
        epsilon=None, decay=0.0, amsgrad=False
    )
    model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=adam)

    # Print model details
    print('Model layers:', model.layers)        # list of layers
    print('Model config:', model.get_config())  # config dictionary
    print('Model summary:')
    print(model.summary())                      # detailed summary

In [ ]:
import matplotlib
matplotlib.use('TkAgg')   # Use TkAgg backend for matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import itertools
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, mean_absolute_error, r2_score, accuracy_score


# =======================
# Plot confusion matrix
# =======================
def plot_confusion_matrix(cm, title='', cmap=plt.get_cmap("Blues"), labels=[], save_filename=None):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm*100, interpolation='nearest', cmap=cmap)  # Multiply by 100 for percentage scale
    plt.title(title, fontsize=10)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    # Add class labels to x and y axis
    plt.xticks(tick_marks, labels, rotation=90, size=12)
    plt.yticks(tick_marks, labels, size=12)

    # Add values to each cell
    for i in range(len(tick_marks)):
        for j in range(len(tick_marks)):
            if i != j:
                # Off-diagonal entries: prediction errors
                plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10)
            else:
                # Diagonal entries: correct predictions (highlighted in orange)
                color = 'darkorange'
                plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10, color=color)

    plt.tight_layout()
    plt.ylabel('True label', fontdict={'size':16,})
    plt.xlabel('Predicted label', fontdict={'size':16,})

    # Save confusion matrix plot if filename provided
    if save_filename is not None:
        plt.savefig(save_filename, format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

# =======================
# Calculate confusion matrix
# =======================
def calculate_confusion_matrix(Y, Y_hat, classes):
    n_classes = len(classes)
    conf = np.zeros([n_classes, n_classes])      # Raw confusion matrix
    confnorm = np.zeros([n_classes, n_classes])  # Normalized confusion matrix

    # Fill confusion matrix by comparing true vs predicted labels
    for k in range(0, Y.shape[0]):
        i = list(Y[k,:]).index(1)           # True class index
        j = int(np.argmax(Y_hat[k,:]))      # Predicted class index
        conf[i,j] = conf[i,j] + 1

    # Normalize rows (per-class accuracy)
    for i in range(0, n_classes):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    # Count correct and incorrect predictions
    right = np.sum(np.diag(conf))
    wrong = np.sum(conf) - right
    return confnorm, right, wrong

# =======================
# Calculate per-class accuracy at given SNR
# =======================
def calculate_acc_at1snr_from_cm(cm):
    return np.round(np.diag(cm) / np.sum(cm, axis=1), 3)

# =======================
# Calculate metrics: Accuracy
# =======================
def calculate_metrics(Y, Y_hat):
    Y_true = np.argmax(Y, axis=1)       # Convert one-hot to label index
    Y_pred = np.argmax(Y_hat, axis=1)
    accuracy = accuracy_score(Y_true, Y_pred)

    return accuracy

# =======================
# Calculate accuracy & metrics per SNR
# =======================
def calculate_acc_cm_each_snr(Y, Y_hat, Z, classes=None, save_figure=True, min_snr=0):
    Z_array = Z[:, 0]                # Extract SNR values
    snrs = sorted(list(set(Z_array)))  # Unique SNRs
    acc = np.zeros(len(snrs))          # Store overall accuracy per SNR
    acc_mod_snr = np.zeros((len(classes), len(snrs)))  # Store per-class accuracy

    # Dictionary to store metrics at each SNR
    metrics = {
        'accuracy': []
    }

    i = 0
    for snr in snrs:
        # Select data corresponding to this SNR
        Y_snr = Y[np.where(Z_array == snr)]
        Y_hat_snr = Y_hat[np.where(Z_array == snr)]

        # Compute confusion matrix
        cm, right, wrong = calculate_confusion_matrix(Y_snr, Y_hat_snr, classes)
        accuracy = calculate_metrics(Y_snr, Y_hat_snr)

        # Store metrics
        metrics['accuracy'].append(accuracy)


        # Plot confusion matrix if above threshold SNR
        if snr >= min_snr:
            plot_confusion_matrix(cm, cmap=plt.cm.Blues, labels=classes, save_filename='figure/cm_snr{}.pdf'.format(snr))

        # Overall accuracy for this SNR
        acc[i] = round(1.0 * right / (right + wrong), 3)
        print('Accuracy at %ddb: %.2f%s / (%d + %d)' % (snr, 100*acc[i], '%', right, wrong))
        acc_mod_snr[:, i] = calculate_acc_at1snr_from_cm(cm)
        i += 1

    # Save accuracy results into a file
    fd = open('acc_overall_128k_on_512k_wts.dat', 'wb')
    pickle.dump(('128k', '512k', acc), fd)
    fd.close()

    # Plot Accuracy
    for metric, values in metrics.items():
        plt.figure(figsize=(8, 6))
        plt.plot(snrs, values, label=metric)
        for x, y in zip(snrs, values):
            plt.text(x, y, round(y, 3), ha='center', va='bottom', fontsize=8)
        plt.xlabel("Signal to Noise Ratio")
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} vs SNR")
        plt.legend()
        plt.grid()
        plt.savefig(f'figure/{metric}_vs_snr.pdf', format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()

In [ ]:
import random
import os

# Set Keras backend to TensorFlow
os.environ["KERAS_BACKEND"] = "tensorflow"

# Select which GPU to use (here GPU 0 is visible, others hidden)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Imports for plotting colored line segments
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

import sys, h5py
import tensorflow as tf
import pandas as pd

# Utility function from Keras to convert labels to one-hot encoding
from keras.utils.np_utils import to_categorical

# List of modulation classes used in the dataset (26 total)
classes = [
    'BPSK',        # Binary Phase Shift Keying
    'QPSK',        # Quadrature Phase Shift Keying
    '8PSK',        # 8-level Phase Shift Keying
    '16PSK',       # 16-level Phase Shift Keying
    '32PSK',       # 32-level Phase Shift Keying
    '64PSK',       # 64-level Phase Shift Keying
    '4QAM',        # 4-level Quadrature Amplitude Modulation
    '8QAM',        # 8-level Quadrature Amplitude Modulation
    '16QAM',       # 16-level Quadrature Amplitude Modulation
    '32QAM',       # 32-level Quadrature Amplitude Modulation
    '64QAM',       # 64-level Quadrature Amplitude Modulation
    '128QAM',      # 128-level Quadrature Amplitude Modulation
    '256QAM',      # 256-level Quadrature Amplitude Modulation
    '2FSK',        # 2-level Frequency Shift Keying
    '4FSK',        # 4-level Frequency Shift Keying
    '8FSK',        # 8-level Frequency Shift Keying
    '16FSK',       # 16-level Frequency Shift Keying
    '4PAM',        # 4-level Pulse Amplitude Modulation
    '8PAM',        # 8-level Pulse Amplitude Modulation
    '16PAM',       # 16-level Pulse Amplitude Modulation
    'AM-DSB',      # Amplitude Modulation - Double Sideband
    'AM-DSB-SC',   # Amplitude Modulation - Double Sideband Suppressed Carrier
    'AM-USB',      # Amplitude Modulation - Upper Sideband
    'AM-LSB',      # Amplitude Modulation - Lower Sideband
    'FM',          # Frequency Modulation
    'PM'           # Phase Modulation
]

In [ ]:
## Load training and testing data

#These are changed based on Dataset (RML2016.10a, RML2016.10b are RML2018.01a)

#/HisarMod2019.1
# Open training dataset file (.mat format) with h5py
data1 = h5py.File('Dataset/HisarMod2019.1/Train/train.mat', 'r')
# Extract data array stored under key 'data_save'
train = data1['data_save'][:]
# Rearrange dimensions: move axis 0 to the end (needed for Keras/TensorFlow input format)
train = train.swapaxes(0, 2)

# Open testing dataset file (.mat format)
data2 = h5py.File('Dataset/HisarMod2019.1/Test/test.mat', 'r')
# Extract test data
test = data2['data_save'][:]
# Rearrange dimensions for consistency
test = test.swapaxes(0, 2)

# Add a channel dimension at the end (axis=3)
# This converts data into shape (samples, height, width, channels),
# which is the standard input format for Conv2D in Keras
train = np.expand_dims(train, axis=3)
test = np.expand_dims(test, axis=3)

In [ ]:
## Load and preprocess labels for training and testing data

# -------------------------
# Training labels
# -------------------------
# Read CSV file containing training labels (no header in CSV)
train_labels = pd.read_csv('Dataset/HisarMod2019.1/Train/train_labels1.csv', header=None)
# Convert pandas DataFrame to numpy array
train_labels = np.array(train_labels)
# Convert labels to one-hot encoding for classification
# Example: label '3' becomes [0,0,0,1,0,...]
train_labels = to_categorical(train_labels, num_classes=None)

# -------------------------
# Testing labels
# -------------------------
# Read CSV file containing testing labels
test_labels = pd.read_csv('Dataset/HisarMod2019.1/Test/test_labels1.csv', header=None)
# Convert to numpy array
test_labels = np.array(test_labels)
# Convert testing labels to one-hot encoding
test_labels = to_categorical(test_labels, num_classes=None)

In [ ]:
## Load Signal-to-Noise Ratio (SNR) values for training and testing data

# -------------------------
# Training SNR
# -------------------------
# Read CSV file containing SNR values for each training sample (no header in CSV)
train_snr = pd.read_csv('Dataset/HisarMod2019.1/Train/train_snr.csv', header=None)
# Convert pandas DataFrame to numpy array for easier manipulation
train_snr = np.array(train_snr)

# -------------------------
# Testing SNR
# -------------------------
# Read CSV file containing SNR values for each testing sample
test_snr = pd.read_csv('Dataset/HisarMod2019.1/Test/test_snr.csv', header=None)
# Convert to numpy array
test_snr = np.array(test_snr)

In [ ]:
import pickle
import numpy as np
from numpy import linalg as la
# RML2016.10a dataset loader and preprocessing

maxlen = 128   # maximum sequence length (IQ samples)

# L2 normalization along a given axis
def l2_normalize(x, axis=-1):
    # compute squared norm, take max along axis, then normalize
    y = np.max(np.sum(x ** 2, axis, keepdims=True), axis, keepdims=True)
    return x / np.sqrt(y)

# Normalize each sample (first channel) to unit L2-norm
def norm_pad_zeros(X_train, nsamples):
    print("Pad:", X_train.shape)
    for i in range(X_train.shape[0]):
        # normalize real part (channel 0) by its L2 norm
        X_train[i, :, 0] = X_train[i, :, 0] / la.norm(X_train[i, :, 0], 2)
    return X_train


# Convert I/Q (real, imag) to amplitude and phase representation
def to_amp_phase(X_train, X_val, X_test, nsamples):
    # complex representation: I + jQ
    X_train_cmplx = X_train[:, 0, :] + 1j * X_train[:, 1, :]
    X_val_cmplx = X_val[:, 0, :] + 1j * X_val[:, 1, :]
    X_test_cmplx = X_test[:, 0, :] + 1j * X_test[:, 1, :]

    # amplitude and normalized angle for training set
    X_train_amp = np.abs(X_train_cmplx)
    X_train_ang = np.arctan2(X_train[:, 1, :], X_train[:, 0, :]) / np.pi
    X_train_amp = np.reshape(X_train_amp, (-1, 1, nsamples))
    X_train_ang = np.reshape(X_train_ang, (-1, 1, nsamples))
    X_train = np.concatenate((X_train_amp, X_train_ang), axis=1)
    X_train = np.transpose(np.array(X_train), (0, 2, 1))

    # same processing for validation set
    X_val_amp = np.abs(X_val_cmplx)
    X_val_ang = np.arctan2(X_val[:, 1, :], X_val[:, 0, :]) / np.pi
    X_val_amp = np.reshape(X_val_amp, (-1, 1, nsamples))
    X_val_ang = np.reshape(X_val_ang, (-1, 1, nsamples))
    X_val = np.concatenate((X_val_amp, X_val_ang), axis=1)
    X_val = np.transpose(np.array(X_val), (0, 2, 1))

    # same processing for test set
    X_test_amp = np.abs(X_test_cmplx)
    X_test_ang = np.arctan2(X_test[:, 1, :], X_test[:, 0, :]) / np.pi
    X_test_amp = np.reshape(X_test_amp, (-1, 1, nsamples))
    X_test_ang = np.reshape(X_test_ang, (-1, 1, nsamples))
    X_test = np.concatenate((X_test_amp, X_test_ang), axis=1)
    X_test = np.transpose(np.array(X_test), (0, 2, 1))

    return (X_train, X_val, X_test)


# Load RML2016.10a dataset and prepare train/val/test splits
def load_data(filename=r'E:\RML_2016_10A\data\RML2016.10a_dict.pkl'):
    # load pickle file: dictionary with keys = (modulation, SNR)
    Xd = pickle.load(open(filename, 'rb'), encoding='iso-8859-1')
    # extract list of modulation types and SNR values
    mods, snrs = [sorted(list(set([k[j] for k in Xd.keys()]))) for j in [0, 1]]
    # mods: e.g. ['8PSK','AM-DSB','BPSK',...]

    X = []         # store signal data
    lbl = []       # store labels (mod, snr)
    train_idx = [] # indices for training
    val_idx = []   # indices for validation
    np.random.seed(2016) # reproducibility
    a = 0

    # collect data for each modulation × SNR pair
    for mod in mods:
        for snr in snrs:
            X.append(Xd[(mod, snr)])  # shape (6000, 2, 128)
            for i in range(Xd[(mod, snr)].shape[0]):
                lbl.append((mod, snr))
            # randomly split indices: 600 train, 200 val, rest test
            train_idx += list(np.random.choice(range(a*1000, (a+1)*1000), size=600, replace=False))
            val_idx += list(np.random.choice(list(set(range(a*1000, (a+1)*1000)) - set(train_idx)), size=200, replace=False))
            a += 1

    # stack all signals into one big numpy array
    X = np.vstack(X)
    n_examples = X.shape[0]

    # remaining indices go to test set
    test_idx = list(set(range(0, n_examples)) - set(train_idx) - set(val_idx))

    # shuffle index lists
    np.random.shuffle(train_idx)
    np.random.shuffle(val_idx)
    np.random.shuffle(test_idx)

    # split data
    X_train = X[train_idx]
    X_val   = X[val_idx]
    X_test  = X[test_idx]

    # helper: convert labels to one-hot encoding
    def to_onehot(yy):
        yy1 = np.zeros([len(yy), len(mods)])
        yy1[np.arange(len(yy)), yy] = 1
        return yy1

    # one-hot labels for train/val/test
    Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
    Y_val   = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), val_idx)))
    Y_test  = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

    # convert to amplitude/phase representation
    X_train, X_val, X_test = to_amp_phase(X_train, X_val, X_test, 128)

    # cut or pad to maxlen
    X_train = X_train[:, :maxlen, :]
    X_val   = X_val[:, :maxlen, :]
    X_test  = X_test[:, :maxlen, :]

    # normalize real parts
    X_train = norm_pad_zeros(X_train, maxlen)
    X_val   = norm_pad_zeros(X_val, maxlen)
    X_test  = norm_pad_zeros(X_test, maxlen)

    # shapes check
    print(X_train.shape)
    print(X_val.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_val.shape)
    print(Y_test.shape)

    return (mods, snrs, lbl), (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), (train_idx, val_idx, test_idx)


# Run script directly: load dataset and preprocess
if __name__ == '__main__':
    (mods, snrs, lbl), (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), (train_idx, val_idx, test_idx) = load_data()

In [ ]:
# RML2016.10b dataset loader and preprocessing

def load_data(filename=r'E:\RML_2016_10B\data\RML2016.10b.dat'):
    # Load dataset from pickle file (dictionary: keys=(modulation, SNR), values=(samples))
    Xd = pickle.load(open(filename, 'rb'), encoding='iso-8859-1')
    # Extract list of modulation types and SNR values from dictionary keys
    mods, snrs = [sorted(list(set([k[j] for k in Xd.keys()]))) for j in [0, 1]]

    X = []          # all samples (stacked later)
    lbl = []        # labels: (modulation, snr) for each sample
    train_idx = []  # training indices
    val_idx = []    # validation indices
    np.random.seed(2016)  # reproducibility
    a = 0           # counter to shift index ranges

    # Iterate over all modulation × SNR combinations
    for mod in mods:
        for snr in snrs:
            # Append the samples for this (mod, snr) pair → shape (1000, 2, 128)
            X.append(Xd[(mod, snr)])
            # Create label list (same length as number of samples)
            for i in range(Xd[(mod, snr)].shape[0]):
                lbl.append((mod, snr))
            # Randomly select 3600 training indices from this block of 6000
            train_idx += list(np.random.choice(range(a*6000, (a+1)*6000), size=3600, replace=False))
            # Randomly select 1200 validation indices (disjoint from training set)
            val_idx += list(np.random.choice(list(set(range(a*6000, (a+1)*6000)) - set(train_idx)), size=1200, replace=False))
            a += 1   # move to next block

    # Stack into one big array: shape (220000, 2, 128)
    X = np.vstack(X)
    print(len(lbl))    # total number of labels (should match X.shape[0])

    n_examples = X.shape[0]
    # Remaining indices are assigned to the test set
    test_idx = list(set(range(0, n_examples)) - set(train_idx) - set(val_idx))

    # Shuffle all index lists
    np.random.shuffle(train_idx)
    np.random.shuffle(val_idx)
    np.random.shuffle(test_idx)

    # Split dataset into train, validation, and test sets
    X_train = X[train_idx]
    X_val   = X[val_idx]
    X_test  = X[test_idx]

    # Print counts
    print(len(train_idx))
    print(len(val_idx))
    print(len(test_idx))

    # Print dataset shapes
    print(X_train.shape)
    print(X_val.shape)
    print(X_test.shape)

    # Helper function: convert labels into one-hot vectors
    def to_onehot(yy):
        yy1 = np.zeros([len(yy), len(mods)])  # (#samples, #modulation classes)
        yy1[np.arange(len(yy)), yy] = 1
        return yy1

    # One-hot encode labels for train/val/test
    Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
    Y_val   = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), val_idx)))
    Y_test  = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

    # Print shapes of labels
    print(Y_train.shape)
    print(Y_val.shape)
    print(Y_test.shape)

    # Return everything
    return (mods, snrs, lbl), (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), (train_idx, val_idx, test_idx)


# Run script directly: load dataset and split
if __name__ == '__main__':
    (mods, snrs, lbl), (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), (train_idx, val_idx, test_idx) = load_data()

In [ ]:
# RML2018.01a dataset loader and preprocessing

def sliceX_normalized(from_filename='/media/GOLD_XYZ_OSC.0001_1024.hdf5',
                      slice_len=1024,
                      to_filename='/media/XYZ_1024_2m_norm.hdf5'):
    """
    Normalize raw RML2018.01a dataset slices and save to new HDF5 file.
    Each sample is normalized with min-max scaling to range [-10, 10].
    """
    from_file = h5py.File(from_filename, 'r')
    X = from_file['X']   # raw IQ samples
    Y = from_file['Y']   # one-hot encoded modulation labels
    Z = from_file['Z']   # SNR labels

    # create new HDF5 file for normalized data
    to_file = h5py.File(to_filename, 'w')
    X_slice = to_file.create_dataset(
        name='X',
        shape=(X.shape[0], slice_len, X.shape[2]),
        maxshape=(None, slice_len, X.shape[2]),
        dtype=np.float32,
        compression='gzip',
        chunks=(1, slice_len, X.shape[2])
    )
    Y_slice = to_file.create_dataset(
        name='Y',
        shape=Y.shape,
        maxshape=(None, Y.shape[1]),
        dtype=np.uint8,
        compression='gzip',
        chunks=(1, Y.shape[1])
    )
    Z_slice = to_file.create_dataset(
        name='Z',
        shape=Z.shape,
        dtype=np.int,
        compression='gzip'
    )

    # normalize batch by batch
    batch_size = 4096
    for i in range(int(X.shape[0] / batch_size)):
        # take a slice of data
        batch_X = X[i * batch_size:(i + 1) * batch_size, 0:slice_len, :]

        # normalize each sample inside the batch
        for j in range(batch_X.shape[0]):
            x = batch_X[j]
            n_s0, n_s1 = x.shape[0], x.shape[1]
            x = np.reshape(x, (n_s0 * n_s1,))
            # min-max scaling to [-10, 10]
            x = sklearn.preprocessing.minmax_scale(x, feature_range=(-10, 10))
            x = np.reshape(x, (n_s0, n_s1))
            batch_X[j] = x

        # save normalized data into new HDF5 file
        X_slice[i * batch_size:(i + 1) * batch_size, :, :] = batch_X
        Y_slice[i * batch_size:(i + 1) * batch_size, :] = Y[i * batch_size:(i + 1) * batch_size, :]
        Z_slice[i * batch_size:(i + 1) * batch_size] = Z[i * batch_size:(i + 1) * batch_size]
        print('write batch {}-{}'.format(i * batch_size, (i + 1) * batch_size))

    from_file.close()
    to_file.close()


def subsample_data_2018_tofile(from_filename="/media/XYZ_1024_2m_norm.hdf5",
                               sample_rate=1/12,
                               to_filename="/media/norm_XYZ_1024_512k.hdf5"):
    """
    Subsample the normalized dataset to reduce size.
    Keeps `sample_rate` fraction of samples per SNR group.
    """
    f = h5py.File(from_filename, 'r')
    X = f['X']
    Y = f['Y']
    Z = f['Z']

    to_file = h5py.File(to_filename, 'w')
    n_subsample = int(X.shape[0] * sample_rate)

    # allocate new datasets for subsampled data
    X_slice = to_file.create_dataset(
        name='X',
        shape=(n_subsample, X.shape[1], X.shape[2]),
        maxshape=(None, X.shape[1], X.shape[2]),
        dtype=np.float32,
        compression='gzip',
        chunks=(1, X.shape[1], X.shape[2])
    )
    Y_slice = to_file.create_dataset(
        name='Y',
        shape=(n_subsample, Y.shape[1]),
        maxshape=(None, Y.shape[1]),
        dtype=np.uint8,
        compression='gzip',
        chunks=(1, Y.shape[1])
    )
    Z_slice = to_file.create_dataset(
        name='Z',
        shape=(n_subsample, 1),
        dtype=np.int,
        compression='gzip'
    )

    snr_count = 26  # total number of SNR levels
    batch_size = int(X.shape[0] / snr_count)
    n_slice = int(batch_size * sample_rate)

    # random sample from each SNR group
    for i in range(snr_count):
        print('subsample the snr {}'.format(i))
        batch_X = X[i * batch_size:(i + 1) * batch_size]
        batch_Y = Y[i * batch_size:(i + 1) * batch_size]
        batch_Z = Z[i * batch_size:(i + 1) * batch_size]

        np.random.seed(2016)
        rand_idx = np.random.choice(np.arange(0, batch_size), size=n_slice, replace=False)

        X_slice[i * n_slice:(i + 1) * n_slice, :, :] = batch_X[rand_idx]
        Y_slice[i * n_slice:(i + 1) * n_slice, :] = batch_Y[rand_idx]
        Z_slice[i * n_slice:(i + 1) * n_slice] = batch_Z[rand_idx]

    to_file.close()
    f.close()
    print('subsample complete. total samples: {}'.format(n_subsample))


def load_data_2018(from_filename="/media/XYZ.0001_0512_NORM.hdf5"):
    """
    Load normalized RML2018 dataset from file.
    Returns X (IQ samples), Y (labels), Z (SNRs).
    """
    f = h5py.File(from_filename, 'r')
    X = f['X'][:]
    Y = f['Y'][:]
    Z = f['Z'][:]
    f.close()
    return X, Y, Z


def data_analyse(filename=None):
    """
    Analyse dataset min/max values for sanity check (before & after normalization).
    """
    import pandas as pd
    X, _, _ = load_data_2018(from_filename=filename)
    pX = pd.Panel(X)
    pX_max = pX.max()
    print('max(I,Q) of the origin data is {}\r'.format(pX_max.max(axis=1)))
    pX_min = pX.min()
    print('min(I) of the origin data is {}\r'.format(pX_min.min(axis=1)))
    print('=========================================================')

    Xn, _, _ = load_data_2018(from_filename=filename)
    pXn = pd.Panel(Xn)
    pXn_max = pXn.max()
    print('max(I,Q) of the normalized data is {}'.format(pXn_max.max(axis=1)))
    pXn_min = pXn.min()
    print('min(I) of the normalized data is {}'.format(pXn_min.min(axis=1)))


def data_structure(filename='/media/XYZ_1024_512k.hdf5'):
    """
    Inspect HDF5 dataset structure, print shapes and distributions of X, Y, Z.
    Also visualize distributions of SNRs and modulation labels.
    """
    f = h5py.File(filename, 'r')
    print('hdf5 keys include 3 DataSet:')
    for k in f.keys():
        print(k)

    print('each dataset shape:')
    print('X:{}, Y:{}, Z:{}'.format(f['X'].shape, f['Y'].shape, f['Z'].shape))

    print('each dataset type:')
    print(type(f['X']), type(f['Y']), type(f['Z']))

    print('each dataset element shape:')
    print('X[i]:{}, Y[i]:{}, Z[i]:{}'.format(f['X'][0].shape, f['Y'][0].shape, f['Z'][0].shape))

    print('Y element like:')
    print(f['Y'][0])

    snrs = sorted(list(set(f['Z'][:, 0])))
    print('Z element include:')
    print(snrs)

    # visualize SNR distribution
    Z = f['Z']
    lenZ = int(4096 * 24 / 32)
    plt.figure()
    plt.title('Z snr distribute')
    plt.plot(np.arange(lenZ), Z[0:lenZ])
    plt.show()

    # visualize modulation distribution
    Y = f['Y']
    sumY = np.sum(Y, axis=0)
    print(sumY)
    Y_not_onehot = np.argmax(Y, axis=1)
    print(Y_not_onehot)
    lenY = Y_not_onehot.shape[0]
    plt.figure()
    plt.title('Y mod distribute')
    plt.plot(np.arange(lenY), Y_not_onehot[0:lenY])
    plt.show()

    # inspect one sample signal
    oneX = f['X'][0]
    print(oneX)
    oneX = np.power(oneX, 2)
    print(oneX)
    oneX = np.sqrt(oneX[:, 0] + oneX[:, 1])
    print(oneX)
    print(np.sum(oneX))

    f.close()


if __name__ == '__main__':
    # Example: subsample dataset to 1/12th size and save
    subsample_data_2018_tofile(from_filename=r"\GOLD_XYZ_OSC.0001_1024.hdf5",
                               sample_rate=1/12,
                               to_filename=r"\XYZ_1024_1_12.hdf5")
    pass

In [ ]:
# ===========================
# Split dataset into training, validation, and testing sets
# ===========================

# Total number of examples in the training data
n_examples = train.shape[0]

# Define number of training samples (60% of total)
n_train = int(n_examples * 0.6)

# Define number of validation samples (15% of total)
n_val = int(n_examples * 0.15)

# Randomly select indices for training samples without replacement
train_idx = list(np.random.choice(range(0, n_examples), size=n_train, replace=False))

# Remaining indices are used for validation
val_idx = list(set(range(0, n_examples)) - set(train_idx))

# Shuffle the indices to ensure randomness
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)

# -------------------------
# Prepare training and validation sets
# -------------------------
X_train = train[train_idx]           # Training data samples
Y_train = train_labels[train_idx]    # Corresponding one-hot training labels

X_val = train[val_idx]               # Validation data samples
Y_val = train_labels[val_idx]        # Corresponding one-hot validation labels

# -------------------------
# Testing set
# -------------------------
X_test = test                        # Test data samples
Y_test = test_labels                 # Test labels (one-hot encoded)
Z_test = test_snr                     # SNR values for each test sample

In [ ]:
# Set up some params
nb_epoch = 200     # number of epochs to train on
batch_size = 300  # training batch size

In [ ]:
# ===========================
# Initialize and compile the model
# ===========================

# Create an instance of the ICAMC model
model = ICAMC()

# Compile the model
# - Loss: 'categorical_crossentropy' (for multi-class classification)
# - Metrics: 'accuracy' to monitor performance
# - Optimizer: 'adam', a popular adaptive optimizer
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Plot the model architecture and save it as an image
# - 'show_shapes=True' displays the input/output shape for each layer
plot_model(model, to_file='model.png', show_shapes=True)  # Saves model diagram to 'model.png'

# Print a detailed summary of the model to console
# - Shows layer types, output shapes, and number of parameters
model.summary()

In [ ]:
# ===========================
# Define file path to save model weights
# ===========================
filepath = 'weights/weights.h5'  # Path where trained model weights will be saved

# ===========================
# Record the start time for training
# ===========================
import time
TRS_PROPOSED = time.time()  # Store current time (seconds since epoch) to measure training duration later

In [ ]:
# ===========================
# Train the model
# ===========================

history = model.fit(
    X_train,                 # Training data inputs
    Y_train,                 # Training data labels (one-hot encoded)
    batch_size=batch_size,   # Number of samples per gradient update
    epochs=nb_epoch,         # Number of complete passes through the training dataset
    verbose=2,               # Verbosity mode (2 = one line per epoch)
    validation_data=(X_val, Y_val),  # Data for validation at the end of each epoch
    callbacks=[              # List of callbacks to apply during training
        # --------------------------
        # Save the best model weights based on validation loss
        # --------------------------
        keras.callbacks.ModelCheckpoint(
            filepath,                 # File path to save the model weights
            monitor='val_loss',        # Metric to monitor (validation loss)
            verbose=1,                 # Print messages when saving
            save_best_only=True,       # Only save the model if the monitored metric improves
            mode='auto'                # Let Keras decide whether to minimize or maximize the monitored metric
        ),
        # --------------------------
        # Reduce learning rate when validation loss plateaus
        # --------------------------
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',       # Metric to monitor
            factor=0.5,               # Factor to reduce the learning rate by
            verbose=1,                # Print messages when reducing LR
            patince=5,                # Number of epochs with no improvement before reducing LR (note: should be 'patience', check typo)
            min_lr=0.0000001          # Minimum learning rate
        ),
        # --------------------------
        # Early stopping to prevent overfitting
        # --------------------------
        keras.callbacks.EarlyStopping(
            monitor='val_loss',       # Metric to monitor
            patience=5,               # Stop training if no improvement after 5 epochs
            verbose=1,                # Print message when stopping
            mode='auto'               # Let Keras decide whether to minimize or maximize the monitored metric
        )
    ]
)

In [ ]:
# ===========================
# Record the end time and calculate total training duration
# ===========================

TRE_PROPOSED = time.time()           # Capture the current time after training finishes
T_PROPOSED = TRE_PROPOSED - TRS_PROPOSED  # Calculate total training time (seconds) by subtracting start time
T_PROPOSED                           # Display or store the total training duration

In [ ]:
# ===========================
# Evaluate the trained model on the test set
# ===========================

TES_PROPOSED = time.time()  # Record the current time before evaluation (optional, for timing)

# Evaluate the model on test data
# - Returns a list: [loss, accuracy] (because we compiled the model with 'accuracy' metric)
score = model.evaluate(
    X_test,            # Test data inputs
    Y_test,            # Test labels (one-hot encoded)
    verbose=1,         # Print progress bar for evaluation
    batch_size=batch_size  # Number of samples per evaluation step
)

# Print evaluation results: loss and accuracy on test set
print(score)

In [ ]:
show_history(history) # plot loss curve

In [ ]:
def predict(model):
    # ===========================
    # Load the best-trained model weights
    # ===========================
    model.load_weights(filepath)  # Load weights saved during training from 'weights/weights.h5'

    # ===========================
    # Predict labels for the test set
    # ===========================
    test_Y_hat = model.predict(X_test, batch_size=batch_size)
    # 'test_Y_hat' contains predicted probabilities for each class (one-hot-like format)

    # ===========================
    # Compute confusion matrix and overall accuracy
    # ===========================
    cm, right, wrong = calculate_confusion_matrix(Y_test, test_Y_hat, classes)
    # cm: normalized confusion matrix
    # right: number of correct predictions
    # wrong: number of incorrect predictions

    acc = round(1.0 * right / (right + wrong), 4)  # Overall accuracy (0-1 scale)
    print('Overall Accuracy: %.2f%s / (%d + %d)' % (100 * acc, '%', right, wrong))
    # Prints accuracy as a percentage and counts of correct/incorrect predictions

    # ===========================
    # Plot confusion matrix
    # ===========================
    plot_confusion_matrix(
        cm,
        labels=['BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '64PSK',
                '4QAM', '8QAM', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM',
                '2FSK', '4FSK', '8FSK', '16FSK',
                '4PAM', '8PAM', '16PAM',
                'AM-DSB', 'AM-DSB-SC', 'AM-USB', 'AM-LSB', 'FM', 'PM'],
        save_filename='figure/total_confusion.png'
    )
    # Saves the confusion matrix as a PNG file for visualization

    # ===========================
    # Calculate accuracy for each SNR and plot metrics vs SNR
    # ===========================
    calculate_acc_cm_each_snr(
        Y_test,          # True labels
        test_Y_hat,      # Predicted labels
        Z_test,          # SNR values for each test sample
        classes,         # List of modulation classes
        min_snr=-18      # Minimum SNR value to plot
    )
    # - Plots overall accuracy, precision, recall, and F1-score vs SNR

In [ ]:
predict(model)  #Computes confusion matrices for different SNR levels

In [ ]:
# ===========================
# Record the end time of testing and calculate total testing duration
# ===========================

TEE_PROPOSED = time.time()           # Capture the current time after test evaluation and prediction
TE_PROPOSED = TEE_PROPOSED - TES_PROPOSED  # Calculate total testing time in seconds